In [1]:
import os
import openai
from IPython.display import Markdown, display

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    text,
)

In [2]:

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
# create city SQL table
table_name = "person"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("first_name", String(50), nullable=False),
    Column("last_name", String(50), nullable=False),
    Column("age", Integer),
)
metadata_obj.create_all(engine)

In [3]:
from sqlalchemy import insert

rows = [
    {"first_name": "John", "last_name": "Doe", "age": 30},
    {"first_name": "Jane", "last_name": "Smith", "age": 25},
    {"first_name": "Alice", "last_name": "Johnson", "age": 35},
]
with engine.connect() as conn:
    conn.execute(insert(city_stats_table), rows)
    conn.commit()


In [4]:
# view current table
sql = "SELECT * FROM person"
with engine.connect() as connection:
    results = connection.execute(text(sql))
    for row in results:
        print(row)


(1, 'John', 'Doe', 30)
(2, 'Jane', 'Smith', 25)
(3, 'Alice', 'Johnson', 35)


In [5]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
llm = OpenAI(temperature=0, model="gpt-4o")
sql_database = SQLDatabase(engine, include_tables=["person"])

In [15]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core import SQLDatabase
from llama_index.core.response.notebook_utils import display_source_node

sql_database = SQLDatabase(engine, include_tables=["person"])

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["person"], return_raw=True
)
results = nl_sql_retriever.retrieve(
    "how old is John Doe"
)
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 058ac4fa-048d-43e5-b414-b55c9e4df6b9<br>**Similarity:** None<br>**Text:** [(30,)]<br>**Metadata:** {'sql_query': "SELECT age\nFROM person\nWHERE first_name = 'John' AND last_name = 'Doe';", 'result': [(30,)], 'col_keys': ['age']}<br>

In [17]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)
response = query_engine.query(
    "Return the top 3 oldest people"
)
print(str(response))

Return Alice Johnson, John Doe, and Jane Smith.


In [11]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["person"], llm=llm
)
query_str = "how old is John Doe"
response = query_engine.query(query_str)
print(response)


John Doe is 30 years old.


In [12]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="person"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [14]:
response = query_engine.query("Which person has the highest age?")
display(Markdown(f"{response}"))

Alice Johnson has the highest age at 35 years old.